<a href="https://colab.research.google.com/github/rogerwzeng/HarvardSCSI104/blob/main/e104_Zeng_Roger_HW01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is from the book: 

**Hands-On Machine Learning with Scikit-Learn, Keras and tensorFlow 2, by Aurelien Geron, O'Reilly, 2019**

Chapter 19 – Training and Deploying TensorFlow Models at Scale

_This notebook contains parts of the sample code in chapter 19._

# Setup
First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.


In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if IS_COLAB or IS_KAGGLE:
    !echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" > /etc/apt/sources.list.d/tensorflow-serving.list
    !curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
    !apt update && apt-get install -y tensorflow-model-server
    %pip install -q -U tensorflow-serving-api

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deploy"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  33443      0 --:--:-- --:--:-- --:--:-- 33443
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,026 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:10 http://s

Once you have installed the GPU card(s) and all the required drivers and libraries,
you can use the nvidia-smi command to check that CUDA is properly installed. It
lists the available GPU cards, as well as processes running on each card:

In [2]:
!nvidia-smi

Fri Jan 27 04:54:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    42W / 400W |      3MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Test for the presence of GPUs from Python code

**Note**: `tf.test.is_gpu_available()` is deprecated. Instead, use `tf.config.list_physical_devices('GPU')`.

In [3]:
#tf.test.is_gpu_available() # deprecated
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
tf.test.is_built_with_cuda()

True

The `list_physical_devices()` function returns the list of all available
GPU devices (just one in this example).11

In [6]:
from tensorflow.python.client.device_lib import list_local_devices

devices = list_local_devices()
devices

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15939285289574986983
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 40215117824
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13584960362285827471
 physical_device_desc: "device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0"
 xla_global_id: 416903419]

### Save/Load a SavedModel
In the following exampple wre will use the standard MNIST model provided as `keras.dataset.mnist`

In [7]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

11490434/11490434 [==============================] - 0s 0us/step


## Distributed Training
Many models can be trained quite well on a single GPU, or even on a CPU. But if
training is too slow, you can try distributing it across multiple GPUs on the same
machine. If that’s still too slow, try using more powerful GPUs, or add more GPUs to
the machine. If your model performs heavy computations (such as large matrix multiplications),
then it will run much faster on powerful GPUs, and you could even try
to use TPUs on Google Cloud AI Platform, which will usually run even faster for such
models. But if you can’t fit any more GPUs on the same machine, and if TPUs aren’t
for you (e.g., perhaps your model doesn’t benefit much from TPUs, or perhaps you
want to use your own hardware infrastructure), then you can try training it across
several servers, each with multiple GPUs (if this is still not enough, as a last resort you
can try adding some model parallelism, but this requires a lot more effort).

In [8]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

We will first create and train a standard 2D convolutional model which could be run on either CPU or GPU

In [10]:
def create_model():
    return keras.models.Sequential([
        keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                            padding="same", input_shape=[28, 28, 1]),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"), 
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(units=64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])

In [11]:
batch_size = 100
model = create_model()
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

Epoch 1/10
550/550 [==============================] - 12s 4ms/step - loss: 1.3434 - accuracy: 0.5520 - val_loss: 0.3747 - val_accuracy: 0.8978
Epoch 2/10
550/550 [==============================] - 2s 4ms/step - loss: 0.5005 - accuracy: 0.8451 - val_loss: 0.2199 - val_accuracy: 0.9382
Epoch 3/10
550/550 [==============================] - 2s 4ms/step - loss: 0.3394 - accuracy: 0.8987 - val_loss: 0.1539 - val_accuracy: 0.9526
Epoch 4/10
550/550 [==============================] - 2s 4ms/step - loss: 0.2645 - accuracy: 0.9221 - val_loss: 0.1137 - val_accuracy: 0.9684
Epoch 5/10
550/550 [==============================] - 2s 4ms/step - loss: 0.2148 - accuracy: 0.9370 - val_loss: 0.0962 - val_accuracy: 0.9726
Epoch 6/10
550/550 [==============================] - 2s 4ms/step - loss: 0.1861 - accuracy: 0.9459 - val_loss: 0.0783 - val_accuracy: 0.9756
Epoch 7/10
550/550 [==============================] - 2s 4ms/step - loss: 0.1647 - accuracy: 0.9517 - val_loss: 0.0722 - val_accuracy: 0.9778
Epoch

TensorFlow comes with a very simple API that takes care of all the complexity
for you: the Distribution Strategies API. To train a Keras model across all available
GPUs (on a single machine, for now) using data parallelism with the mirrored
strategy, create a `MirroredStrategy` object, call its `scope()` method to get a distribution
context, and wrap the creation and compilation of your model inside that context.
Then call the model’s `fit()` method normally.

The following cell lets us choose one of severla options depending on the hardware configuration.

Under the hood, tf.keras is distribution-aware, so in this MirroredStrategy context it
knows that it must replicate all variables and operations across all available GPU
devices. Note that the fit() method will automatically split each training batch
across all the replicas, so it’s important that the batch size be divisible by the number
of replicas. And that’s all! Training will generally be significantly faster than using a
single device, and the code change was really minimal

#### Running on a subset of availablel devices
If you only want to use a subset of all the available GPU devices, you can pass the list
to the MirroredStrategy’s constructor:

`distribution = tf.distribute.MirroredStrategy(["/gpu:0", "/gpu:1"])`

By default, the MirroredStrategy class uses the NVIDIA Collective Communications
Library (NCCL) for the AllReduce mean operation, but you can change it by setting
the `cross_device_ops` argument to an instance of the `tf.distribute.Hierarchical
CopyAllReduce` class, or an instance of the `tf.distribute.ReductionToOneDevice`
class. The default NCCL option is based on the `tf.distribute.NcclAllReduce` class,
which is usually faster, but this depends on the number and types of GPUs, so you
may want to give the alternatives a try.

#### Data parallelism with centralized parameters

If you want to try using data parallelism with centralized parameters, replace the
`MirroredStrategy` with the `CentralStorageStrategy`:

`distribution = tf.distribute.experimental.CentralStorageStrategy()`

You can optionally set the `compute_devices` argument to specify the list of devices
you want to use as workers (by default it will use all available GPUs), and you can
optionally set the `parameter_device` argument to specify the device you want to store
the parameters on (by default it will use the CPU, or the GPU if there is just one).

In [12]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

distribution = tf.distribute.MirroredStrategy()

# Change the default all-reduce algorithm:
distribution = tf.distribute.MirroredStrategy(
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

# Option 1: specify the list of GPUs to use:
#distribution = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
#distribution = tf.distribute.MirroredStrategy(devices=["/gpu:0"])

# Option 2: use the central storage strategy instead:
#distribution = tf.distribute.experimental.CentralStorageStrategy()

# Option 3: if running on Colab and want to use TPU processor
#if IS_COLAB and "COLAB_TPU_ADDR" in os.environ:
#  tpu_address = "grpc://" + os.environ["COLAB_TPU_ADDR"]
#else:
#  tpu_address = ""
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
#tf.config.experimental_connect_to_cluster(resolver)
#tf.tpu.experimental.initialize_tpu_system(resolver)
#distribution = tf.distribute.experimental.TPUStrategy(resolver)

with distribution.scope():
    model = create_model()
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

In [ ]:
batch_size = 100 # must be divisible by the number of workers
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

Epoch 1/10


2023-01-26 17:17:25.455915: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


550/550 [==============================] - ETA: 0s - loss: 1.3348 - accuracy: 0.5830

2023-01-26 17:19:16.393729: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


550/550 [==============================] - 116s 210ms/step - loss: 1.3348 - accuracy: 0.5830 - val_loss: 0.3170 - val_accuracy: 0.9134
Epoch 2/10
550/550 [==============================] - ETA: 0s - loss: 0.4206 - accuracy: 0.8719

...

550/550 [==============================] - 119s 217ms/step - loss: 0.1402 - accuracy: 0.9595 - val_loss: 0.0660 - val_accuracy: 0.9818
Epoch 9/10
550/550 [==============================] - ETA: 0s - loss: 0.1307 - accuracy: 0.9620

2023-01-26 17:38:41.368211: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


550/550 [==============================] - 120s 218ms/step - loss: 0.1307 - accuracy: 0.9620 - val_loss: 0.0612 - val_accuracy: 0.9818
Epoch 10/10
550/550 [==============================] - ETA: 0s - loss: 0.1213 - accuracy: 0.9652

2023-01-26 17:40:41.439247: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


550/550 [==============================] - 120s 218ms/step - loss: 0.1213 - accuracy: 0.9652 - val_loss: 0.0601 - val_accuracy: 0.9828


If you have multiple GPUs you run the experiment on one and then 2 or more GPUs. With more GPUs, you would see a speedup compared to the training on one GPU.

Once you have finished training your model, you can use it to make predictions efficiently:
call the `predict()` method, and it will automatically split the batch across all
replicas, making predictions in parallel (again, the batch size must be divisible by the
number of replicas). 

If you call the model’s `save()` method, it will be saved as a regular
model, not as a mirrored model with multiple replicas. So when you load it, it will
run like a regular model, on a single device (by default GPU 0, or the CPU if there are
no GPUs). 
If you want to load a model and run it on all available devices, you must
call `keras.models.load_model()` within a distribution context, like:

`with distribution.scope():
    mirrored_model = keras.models.load_model("my_mnist_model.h5")`

In [13]:
model.predict(X_new)

1/1 [==============================] - 1s 1s/step


array([[0.10385561, 0.10114489, 0.09581841, 0.0978876 , 0.09493201,
        0.1048363 , 0.09678087, 0.10308881, 0.09845111, 0.10320438],
       [0.10551135, 0.10397926, 0.0933008 , 0.09837992, 0.09383032,
        0.1024676 , 0.09671868, 0.10909896, 0.09808642, 0.09862671],
       [0.10361055, 0.1010047 , 0.09801278, 0.09792618, 0.09564402,
        0.10135707, 0.09678063, 0.10341027, 0.09996779, 0.10228598]],
      dtype=float32)

Custom training loop:

In [14]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

K = keras.backend

distribution = tf.distribute.MirroredStrategy()

with distribution.scope():
    model = create_model()
    optimizer = keras.optimizers.SGD()

with distribution.scope():
    dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat().batch(batch_size)
    input_iterator = distribution.make_dataset_iterator(dataset)
    
@tf.function
def train_step():
    def step_fn(inputs):
        X, y = inputs
        with tf.GradientTape() as tape:
            Y_proba = model(X)
            loss = K.sum(keras.losses.sparse_categorical_crossentropy(y, Y_proba)) / batch_size

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        return loss

    per_replica_losses = distribution.experimental_run(step_fn, input_iterator)
    mean_loss = distribution.reduce(tf.distribute.ReduceOp.SUM,
                                    per_replica_losses, axis=None)
    return mean_loss

n_epochs = 10
with distribution.scope():
    input_iterator.initialize()
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))
        for iteration in range(len(X_train) // batch_size):
            print("\rLoss: {:.3f}".format(train_step().numpy()), end="")
        print()

Instructions for updating:
Use the iterator's `initializer` property instead.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Instructions for updating:
use run() instead


Epoch 1/10
Loss: 0.409
Epoch 2/10
Loss: 0.319
Epoch 3/10
Loss: 0.293
Epoch 4/10
Loss: 0.283
Epoch 5/10
Loss: 0.278
Epoch 6/10
Loss: 0.276
Epoch 7/10
Loss: 0.274
Epoch 8/10
Loss: 0.265
Epoch 9/10
Loss: 0.255
Epoch 10/10
Loss: 0.249


## Training across multiple servers

A TensorFlow cluster is a group of TensorFlow processes running in parallel, usually on different machines, and talking to each other to complete some work, for example training or executing a neural network. Each TF process in the cluster is called a "task" (or a "TF server"). It has an IP address, a port, and a type (also called its role or its job). The type can be `"worker"`, `"chief"`, `"ps"` (parameter server) or `"evaluator"`:
* Each **worker** performs computations, usually on a machine with one or more GPUs.
* The **chief** performs computations as well, but it also handles extra work such as writing TensorBoard logs or saving checkpoints. There is a single chief in a cluster, typically the first worker (i.e., worker #0).
* A **parameter server** (ps) only keeps track of variable values, it is usually on a CPU-only machine.
* The **evaluator** obviously takes care of evaluation. There is usually a single evaluator in a cluster.

The set of tasks that share the same type is often called a "job". For example, the "worker" job is the set of all workers.

To start a TensorFlow cluster, you must first define it. This means specifying all the tasks (IP address, TCP port, and type). For example, the following cluster specification defines a cluster with 3 tasks (2 workers and 1 parameter server). It's a dictionary with one key per job, and the values are lists of task addresses:

In [15]:
cluster_spec = {
    "worker": [
        "machine-a.example.com:2222",  # /job:worker/task:0
        "machine-b.example.com:2222"   # /job:worker/task:1
    ],
    "ps": ["machine-c.example.com:2222"] # /job:ps/task:0
}

Every task in the cluster may communicate with every other task in the server, so make sure to configure your firewall to authorize all communications between these machines on these ports (it's usually simpler if you use the same port on every machine).

When a task is started, it needs to be told which one it is: its type and index (the task index is also called the task id). A common way to specify everything at once (both the cluster spec and the current task's type and id) is to set the `TF_CONFIG` environment variable before starting the program. It must be a JSON-encoded dictionary containing a cluster specification (under the `"cluster"` key), and the type and index of the task to start (under the `"task"` key). For example, the following `TF_CONFIG` environment variable defines the same cluster as above, with 2 workers and 1 parameter server, and specifies that the task to start is worker #1:

In [16]:
import os
import json

os.environ["TF_CONFIG"] = json.dumps({
    "cluster": cluster_spec,
    "task": {"type": "worker", "index": 1}
})
os.environ["TF_CONFIG"]

'{"cluster": {"worker": ["machine-a.example.com:2222", "machine-b.example.com:2222"], "ps": ["machine-c.example.com:2222"]}, "task": {"type": "worker", "index": 1}}'

Some platforms (e.g., Google Cloud ML Engine) automatically set this environment variable for you.

TensorFlow's `TFConfigClusterResolver` class reads the cluster configuration from this environment variable:

In [17]:
import tensorflow as tf

resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
resolver.cluster_spec()

ClusterSpec({'ps': ['machine-c.example.com:2222'], 'worker': ['machine-a.example.com:2222', 'machine-b.example.com:2222']})

In [18]:
resolver.task_type

'worker'

In [19]:
resolver.task_id

1

Now let's run a simpler cluster with just two worker tasks, both running on the local machine. We will use the `MultiWorkerMirroredStrategy` to train a model across these two tasks.

The first step is to write the training code. As this code will be used to run both workers, each in its own process, we write this code to a separate Python file, `my_mnist_multiworker_task.py`. The code is relatively straightforward, but there are a couple important things to note:
* We create the `MultiWorkerMirroredStrategy` before doing anything else with TensorFlow.
* Only one of the workers will take care of logging to TensorBoard and saving checkpoints. As mentioned earlier, this worker is called the *chief*, and by convention it is usually worker #0.

In [20]:
%%writefile my_mnist_multiworker_task.py

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time

# At the beginning of the program
distribution = tf.distribute.MultiWorkerMirroredStrategy()

resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
print("Starting task {}{}".format(resolver.task_type, resolver.task_id))

# Only worker #0 will write checkpoints and log to TensorBoard
if resolver.task_id == 0:
    root_logdir = os.path.join(os.curdir, "my_mnist_multiworker_logs")
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    run_dir = os.path.join(root_logdir, run_id)
    callbacks = [
        keras.callbacks.TensorBoard(run_dir),
        keras.callbacks.ModelCheckpoint("my_mnist_multiworker_model.h5",
                                        save_best_only=True),
    ]
else:
    callbacks = []

# Load and prepare the MNIST dataset
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis] / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

with distribution.scope():
    model = keras.models.Sequential([
        keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                            padding="same", input_shape=[28, 28, 1]),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"), 
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(units=64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
          epochs=10, callbacks=callbacks)

Writing my_mnist_multiworker_task.py


In a real world application, there would typically be a single worker per machine, but in this example we're running both workers on the same machine, so they will both try to use all the available GPU RAM (if this machine has a GPU), and this will likely lead to an Out-Of-Memory (OOM) error. To avoid this, we could use the `CUDA_VISIBLE_DEVICES` environment variable to assign a different GPU to each worker. Alternatively, we can simply disable GPU support, like this:

In [21]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

We are now ready to start both workers, each in its own process, using Python's `subprocess` module. Before we start each process, we need to set the `TF_CONFIG` environment variable appropriately, changing only the task index:

In [22]:
import subprocess

cluster_spec = {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]}

for index, worker_address in enumerate(cluster_spec["worker"]):
    os.environ["TF_CONFIG"] = json.dumps({
        "cluster": cluster_spec,
        "task": {"type": "worker", "index": index}
    })
    subprocess.Popen("python my_mnist_multiworker_task.py", shell=True)

That's it! Our TensorFlow cluster is now running, but we can't see it in this notebook because it's running in separate processes (but if you are running this notebook in Jupyter, you can see the worker logs in Jupyter's server logs).

Since the chief (worker #0) is writing to TensorBoard, we use TensorBoard to view the training progress. Run the following cell, then click on the settings button (i.e., the gear icon) in the TensorBoard interface and check the "Reload data" box to make TensorBoard automatically refresh every 30s. Once the first epoch of training is finished (which may take a few minutes), and once TensorBoard refreshes, the SCALARS tab will appear. Click on this tab to view the progress of the model's training and validation accuracy.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_mnist_multiworker_logs --port=6006

That's it! Once training is over, the best checkpoint of the model will be available in the `my_mnist_multiworker_model.h5` file. You can load it using `keras.models.load_model()` and use it for predictions, as usual:

In [29]:
from tensorflow import keras

model = keras.models.load_model("my_mnist_multiworker_model.h5")
Y_pred = model.predict(X_valid)
np.argmax(Y_pred, axis=-1)

157/157 [==============================] - 1s 3ms/step


array([5, 0, 4, ..., 2, 1, 2])

And that's all for today! Hope you found this useful. 😊

# Exercise Solutions

## 1. to 8.

See Appendix A.

## 9.
_Exercise: Train a model (any model you like) and deploy it to TF Serving or Google Cloud AI Platform. Write the client code to query it using the REST API or the gRPC API. Update the model and deploy the new version. Your client code will now query the new version. Roll back to the first version._

Please follow the steps in the <a href="#Deploying-TensorFlow-models-to-TensorFlow-Serving-(TFS)">Deploying TensorFlow models to TensorFlow Serving</a> section above.

# 10.
_Exercise: Train any model across multiple GPUs on the same machine using the `MirroredStrategy` (if you do not have access to GPUs, you can use Colaboratory with a GPU Runtime and create two virtual GPUs). Train the model again using the `CentralStorageStrategy `and compare the training time._

Please follow the steps in the [Distributed Training](#Distributed-Training) section above.

# 11.
_Exercise: Train a small model on Google Cloud AI Platform, using black box hyperparameter tuning._

Please follow the instructions on pages 716-717 of the book. You can also read [this documentation page](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview) and go through the example in this nice [blog post](https://towardsdatascience.com/how-to-do-bayesian-hyper-parameter-tuning-on-a-blackbox-model-882009552c6d) by Lak Lakshmanan.